In [ ]:
import sys
import os
sys.path.append(r"D:\Program Files\DIgSILENT\PowerFactory 2021 SP2\Python\3.9")
import powerfactory as pf 
from PIL import Image
import matplotlib.pyplot as plt 
import pandas as pd
from tqdm import tqdm
import time
import numpy as np 
app=pf.GetApplication() 
app.Show() 

In [ ]:
import powerfactory as pf
import numpy as np
import random
import matplotlib.pyplot as plt
import pandas as pd

alpha = 5
def penalti_Teg(tegangan):
    if tegangan is None:
        return 10
    elif 0.95 <=  tegangan <= 1.05:
        return abs(1-tegangan)
    else:
        return np.exp(alpha * abs(1-tegangan))

def penalti_NCPI(ncpi):
    if ncpi is None:
        return 10
    elif ncpi <= 1:
        return 0
    else:
        return np.exp(alpha * (ncpi-1))

def obj_voltage():
    terminals = app.GetCalcRelevantObjects("*.ElmTerm")
    Tegangan_All = []
    for term in terminals:
        try:
            nilai_tegangan = term.GetAttribute("m:u")
            Tegangan_All.append(nilai_tegangan)
        except:
            Tegangan_All.append(None)
    faktor_penalti = [penalti_Teg(v) for v in Tegangan_All]
    rata_penalti = np.average(faktor_penalti)
    rata_tegangan = np.mean([v for v in Tegangan_All if v is not None])
    return rata_penalti, rata_tegangan

def obj_NCPI():
    lines = app.GetCalcRelevantObjects("*.ElmLne")
    penalty_list = []
    for line in lines:
        if line.GetAttribute("outserv") == 0:
            try:
                Z = line.GetAttribute("Z1")
                X = line.GetAttribute("X1")
                bus1 = line.GetAttribute("bus1")
                bus2 = line.GetAttribute("bus2")
                V1_kV = bus1.GetAttribute("m:Ul")
                V1_pu = bus1.GetAttribute("m:u")
                V2_pu = bus2.GetAttribute("m:u")
                Q_MVar = line.GetAttribute("m:Q:bus2")
                P_bus2 = line.GetAttribute("m:P:bus2")

                if None in [Z, X, V1_kV, V1_pu, V2_pu, Q_MVar, P_bus2]:
                    continue
                if V1_kV == 0 or X == 0 or V1_pu == 0 or V2_pu == 0:
                    continue

                Zbase = (150 ** 2) / 74.118
                Z_pu = Z / Zbase
                X_pu = X / Zbase
                Q_pu_fvsi = abs(Q_MVar) / 74.118
                P_pu_bus2 = P_bus2 / 74.118

                NCPI_term = (4 * (Z_pu ** 4) * (P_pu_bus2 ** 2)) / ((X_pu ** 2) * (V1_pu ** 4))
                NCPI_2 = (4 * (Z_pu ** 2) * Q_pu_fvsi) / ((V1_pu ** 2) * X_pu)
                NCPI = (NCPI_2 + NCPI_term)
                penalti = penalti_NCPI(NCPI)
                penalty_list.append(penalti)
            except:
                continue

    if not penalty_list:
        return 999
    return np.average(penalty_list)

def LFnormal():
    global v_base, ncpibase
    app.ResetCalculation()
    ldf = app.GetFromStudyCase("ComLdf")
    ldf.Execute()
    v_base, _ = obj_voltage()
    ncpibase = obj_NCPI()

#target buses
class CapacitorManager:
    def __init__(self):
        self.target_names = [
            "GI ANGGREK/II", "GI ISIMU/II", "GI BOTUPINGGE/II", "GI GOBAR/II", 
            "GI TANJUNG KARANG/II", "Tolinggula/II", "GI TILAMUTA/II", 
            "GI MARISA/II", "GI PTBJA/II", "GI PLTG MARISA/II"
        ]
        self.terminals = app.GetCalcRelevantObjects("*.ElmTerm")
        self.current_cap = None
        self.current_cub = None
    
    def get_bus_by_index(self, bus_idx):
        if not (0 <= bus_idx < len(self.target_names)):
            return None
        
        target_name = self.target_names[bus_idx].upper().strip()
        for term in self.terminals:
            if term.loc_name.strip().upper() == target_name:
                return term
        return None
    
    def install_capacitor(self, bus_idx, capacity):

        try:
            # Cleanup previous
            self.cleanup()
            
            bus = self.get_bus_by_index(bus_idx)
            if not bus:
                return False
            
            # Find or create cubicle
            all_cubes = bus.GetContents('*.StaCubic')
            empty_cub = None
            
            for cub in all_cubes:
                if cub.obj_id is None:
                    empty_cub = cub
                    break
            
            if empty_cub is None:
                empty_cub = bus.CreateObject('StaCubic')
                empty_cub.loc_name = "cub_eval"
            
            # Create capacitor
            cap = bus.GetParent().CreateObject('ElmShnt')
            cap.bus1 = empty_cub
            cap.ushnm = bus.uknom
            # cap.SetAttribute("uhsnm", bus.uknom)
            # cap.SetAttribute("qtotn", capacity)
            cap.SetAttribute("shtype", 2)
            cap.SetAttribute("qcapn", capacity)
            cap.loc_name = f"Cap_Eval_{capacity:.1f}"
            
            self.current_cap = cap
            self.current_cub = empty_cub
            return True
            
        except Exception as e:
            print(f"Error installing capacitor: {e}")
            return False
    
    def cleanup(self):
        try:
            if self.current_cap:
                self.current_cap.Delete()
                self.current_cap = None
            if self.current_cub and self.current_cub.loc_name == "cub_eval":
                self.current_cub.Delete()
                self.current_cub = None
        except:
            pass

def evaluate_solution_pure(cap_manager, bus_idx, capacity, w1, w2):
    
    # Validasi bounds
    if not (0 <= bus_idx < 10) or not (5 <= capacity <= 100):
        return 1000.0, np.inf  # Penalty besar untuk solusi invalid
    
    # Install kapasitor
    if not cap_manager.install_capacitor(int(bus_idx), capacity):
        return 1000.0, np.inf
    
    try:
        # Reset dan jalankan load flow
        app.ResetCalculation()
        ldf = app.GetFromStudyCase("ComLdf")
        err = ldf.Execute()
        
        if err != 0:
            return 1000.0, np.inf
        
        # Hitung objective functions
        penalty_voltage, avg_voltage = obj_voltage()
        penalty_ncpi = obj_NCPI()
        
        # Fitness deterministik
        if v_base > 0 and ncpibase > 0:
            fitness = w1 * (penalty_voltage / v_base) + w2 * (penalty_ncpi / ncpibase)
        else:
            fitness = w1 * penalty_voltage + w2 * penalty_ncpi
        
        return fitness, avg_voltage
        
    except Exception as e:
        print(f"Error in evaluation: {e}")
        return 1000.0, np.inf
    
    finally:
        # Cleanup
        cap_manager.cleanup()


list_iter = []
def run_pure_pso():
    global v_base, ncpibase
    
    # Hitung baseline
    LFnormal()
    app.ClearOutputWindow()
    print(f"Baseline - Voltage penalty: {v_base:.6f}, NCPI penalty: {ncpibase:.6f}")
    
    # Parameter PSO
    pop_size = 30
    max_iter = 100
    w_max, w_min = 0.9, 0.2
    c1_initial, c2_initial = 2.5, 1.5
    c1_final, c2_final = 1.5, 2.5
    w1, w2 = 0.5, 0.5
    
    # Bounds
    bus_min, bus_max = 0, 9
    cap_min, cap_max = 5, 100
    
    # manager initialization
    cap_manager = CapacitorManager()
    
    # population initialization
    positions = np.zeros((pop_size, 2))
    velocities = np.zeros((pop_size, 2))
    
    # initialization random positions 
    positions[:, 0] = np.random.uniform(bus_min, bus_max, pop_size)
    positions[:, 1] = np.random.uniform(cap_min, cap_max, pop_size)
    
    # velocity initialization
    velocities[:, 0] = np.random.uniform(-1, 1, pop_size)  # Bus velocity
    velocities[:, 1] = np.random.uniform(-10, 10, pop_size)  # Capacity velocity
    
    # Personal best
    pbest_positions = positions.copy()
    pbest_scores = np.full(pop_size, float('inf'))
    
    # Global best
    gbest_position = None
    gbest_score = float('inf')
    gbest_history = []
    
    print("Starting Pure PSO Optimization...")
    print("=" * 50)
    
    for iteration in range(max_iter):
        w = w_max - (iteration / max_iter) * (w_max - w_min)
        c1 = c1_initial + (c1_final - c1_initial) * (iteration / max_iter)
        c2 = c2_initial + (c2_final - c2_initial) * (iteration / max_iter)
        print(f"Iterasi {iteration + 1}/{max_iter}")
                
        for i in range(pop_size):
            bus_continuous = np.clip(positions[i, 0], bus_min, bus_max)
            bus_idx = int(np.round(bus_continuous))
            bus_idx = np.clip(bus_idx, bus_min, bus_max)
            
            capacity = np.clip(positions[i, 1], cap_min, cap_max)
            
            # fitness evaluation
            fitness, avg_voltage = evaluate_solution_pure(cap_manager, bus_idx, capacity, w1, w2)
            
            # Update personal best
            if fitness < pbest_scores[i]:
                pbest_scores[i] = fitness
                pbest_positions[i] = positions[i].copy()
                
                # Update global best
                if fitness < gbest_score:
                    gbest_score = fitness
                    gbest_position = positions[i].copy()
                    gbest_voltage = avg_voltage
                    bus_name = cap_manager.target_names[bus_idx]
                    print(f"  * New Global Best: {bus_name}, {capacity:.2f} Mvar, fitness: {fitness:.6f}, tegangan sistem: {avg_voltage:.6f}")
        
        gbest_history.append(gbest_score)
        
        for i in range(pop_size):
            if gbest_position is not None:
                for d in range(2):  # Untuk setiap dimensi
                    r1 = random.random()
                    r2 = random.random()
                    
                    velocities[i, d] = (
                        w * velocities[i, d] +
                        c1 * r1 * (pbest_positions[i, d] - positions[i, d]) +
                        c2 * r2 * (gbest_position[d] - positions[i, d])
                    )
                    
                    # Update position
                    positions[i, d] += velocities[i, d]
                    
                    # Boundary handling 
                    if d == 0:  # Bus index
                        positions[i, d] = np.clip(positions[i, d], bus_min, bus_max)
                    else:  # Capacity
                        positions[i, d] = np.clip(positions[i, d], cap_min, cap_max)
        
        print(f"  Best fitness: {gbest_score:.6f}")
        
    
    # Cleanup final
    cap_manager.cleanup()
    
    # Final result
    if gbest_position is not None:
        optimal_bus_idx = int(np.round(np.clip(gbest_position[0], bus_min, bus_max)))
        optimal_capacity = np.clip(gbest_position[1], cap_min, cap_max)
        optimal_bus_name = cap_manager.target_names[optimal_bus_idx]
        
        print("\n" + "=" * 50)
        print("HASIL OPTIMASI PURE PSO")
        print("=" * 50)
        print(f"Bus optimal      : {optimal_bus_name}")
        print(f"Kapasitas optimal: {optimal_capacity:.2f} Mvar")
        print(f"Fitness total    : {gbest_score:.6f}")
        print(f"Iterasi selesai  : {len(gbest_history)}")
        print(f"Rata-rata Tegangan Sistem (pu) : {gbest_voltage:.6f}")
        
        # Convergence curve plot
        plt.figure(figsize=(10, 6))
        plt.plot(range(1, len(gbest_history)+1), gbest_history, 'b-o', linewidth=2)
        #plt.title("Konvergensi Pure PSO untuk Optimal Capacitor Placement")
        plt.xlabel("Iterasi")
        plt.ylabel("Nilai Fitness")
        plt.grid(True, alpha=0.3)
        plt.tight_layout()
        plt.show()
        
        return optimal_bus_name, optimal_capacity, gbest_score, gbest_history
    else:
        print("Optimasi gagal - tidak ada solusi valid")
        return None, None, float('inf')

def diagnose_stagnation():
    global v_base, ncpibase
    
    LFnormal()
    cap_manager = CapacitorManager()
    
    test_solutions = [
        (0, 20), (1, 30), (2, 40), (3, 50), (4, 60),
        (5, 25), (6, 35), (7, 45), (8, 55), (9, 65)
    ]
    
    fitness_values = []
    for bus_idx, capacity in test_solutions:
        fitness,avg_voltage = evaluate_solution_pure(cap_manager, bus_idx, capacity, 0.7, 0.3)
        fitness_values.append(fitness)
        bus_name = cap_manager.target_names[bus_idx]
    
    fitness_range = max(fitness_values) - min(fitness_values)
    fitness_std = np.std(fitness_values)
    
    cap_manager.cleanup()
    return fitness_values

# Run optimation
app.GetFromStudyCase("ComLdf").Execute()
_,avg_voltage = obj_voltage()
print(f"Rata-rata tegangan sistem (pu) sebelum optimasi: {avg_voltage:.6f}")
np.random.seed(42)
random.seed(42)
v_base = 0.0
ncpibase = 0.0
app.ResetCalculation()
app.Hide()
fitness_values = diagnose_stagnation()
result_bus_name, optimal_cap, gbest_score, gbest_history = run_pure_pso()
df_gbest_history = pd.DataFrame({"Iterasi" : len(gbest_history),
                                 "Fitness Value": gbest_history})
path = "D:/Kuliah/Bismillah TA/BISMILLAH/Percobaan/SIDANG/4_COMPENSATED/HASIL_RUN_PSO_2"
df_gbest_history.to_excel(f"{path}/SIMULASI BARU_10.xlsx", index=False)
app.Show()
print(f"\nHasil akhir menunjukkan bus optimal adalah: {result}, dengan kapasitas optimal sebesar {optimal_cap}")
        
